In [15]:
# This cell is added by sphinx-gallery
# It can be customized to whatever you like
%matplotlib inline

In [16]:
import pennylane as qml
from pennylane import numpy as np

np.random.seed(42)

In [17]:
num_qubits = 4
num_layers = 2
dev = qml.device("default.qubit", wires=num_qubits, shots=1000)

In [18]:
def hardware_efficient_ansatz(params):
    """A random variational quantum circuit based on the hardware efficient ansatz. 
    There are no measurements and it is to be used within the global or local circuits

    Args:
        params (array[array[float]]): array of parameters of dimension (num_layers, num_qubits) containing the rotation angles

    Returns:
        not sure... see pennylane documentation
    """

    # Relevant parameters
    assert(len(np.shape(params)) == 2)      # check proper dimensions of params
    num_layers = np.shape(params)[0]        # np.shape(params) = (num_layers, num_qubits)
    num_qubits = np.shape(params)[1]
    
    # Generating the gate sequence from randomly applying RX, RY or RZ with the corresponding rotation angle
    gate_set = [qml.RX, qml.RY, qml.RZ]
    random_gate_sequence = [[np.random.choice(gate_set) for _ in range(num_qubits)] for _ in range(num_layers)]

    # Initial rotations on all qubits
    for i in range(num_qubits):             # rotate all qubits
        qml.RY(np.pi / 4, wires=i)          # "to prevent X, Y , or Z from being an especially preferential 
                                            # direction with respect to gradients."

    # Repeating a layer structure
    for l in range(num_layers):
        # Single random gate layer (single qubit rotations)
        for i in range(num_qubits):
            random_gate_sequence[l][i](params[l][i], wires=i)
        # Nearest neighbour controlled phase gates
        if num_qubits > 1:                          # no entangling gates if using a single qubit
            qml.broadcast(qml.CZ, wires=range(num_qubits), pattern="ring")

In [19]:
@qml.qnode(dev)
def global_circuit(params):
    assert(len(np.shape(params)) == 2)      # check proper dimensions of params
    num_qubits = np.shape(params)[1]        # np.shape(params) = (num_layers, num_qubits)

    hardware_efficient_ansatz(params)
    # Objective operator H = Z_1 Z_2 ... Z_n
    H = qml.PauliZ(0)
    for qubit in range(num_qubits-1):
        H = H @ qml.PauliZ(qubit + 1)
    return qml.expval(H)

@qml.qnode(dev)
def local_circuit(params):
    hardware_efficient_ansatz(params)
    # Objective operator H = Z_1 Z_2
    return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1))

In [20]:
def cost_global(params):
    return global_circuit(params)

def cost_local(params):
    return local_circuit(params)

In [27]:
params = np.random.uniform(0, np.pi, size=(num_layers, num_qubits))
print(cost_local(params))
print(cost_global(params))
print(qml.grad(cost_local)(params)[0][0])
print(qml.grad(cost_global)(params)[0][0])

-0.018
0.066
-0.30599999999999994
0.006999999999999992
